In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore')

#Standart Moving Average
def sma(series, length):
    """
    Calculate the Simple Moving Average (SMA) for a given series.
    """
    return series.rolling(window=length).mean()

#Exponential Moving Average
def ema(series, length):
    """
    Calculate the Exponential Moving Average (EMA) for a given series.
    """
    return series.ewm(span=length, adjust=False).mean()


def Bankery(data):
    df=data.copy()
    close_minus_rolling_min = data['close'] - pd.Series(data['low']).rolling(window=27).min()
    percentage_change = close_minus_rolling_min / (pd.Series(data['high']).rolling(window=27).max() - pd.Series(data['low']).rolling(window=27).min()) * 100

    sma1 = sma(percentage_change, 5)
    sma2 = sma(sma1, 3)

    fundtrend = (3 * sma1 - 2 * sma2 - 50) * 1.032 + 50

    typ = (2 * data['close'] + data['high'] + data['low'] + data['open']) / 5
    lol = pd.Series(data['low']).rolling(window=34).min()
    hoh = pd.Series(data['high']).rolling(window=34).max()
    bullbearline = ema((typ - lol) / (hoh - lol) * 100, 13)
    bankerentry = (fundtrend > bullbearline) & (bullbearline < 25)
    df['Entry'] = (bankerentry==True)
    return df


tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


#Raporlama için kullanılacak başlıklar
Titles = ['Hisse Adı', 'Son Fiyat','Dip Sinyali']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=Interval.in_daily, n_bars=100)
        data = data.reset_index()
        Banker = Bankery(data)
        Banker.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        Banker.set_index('datetime', inplace=True)
        Buy = False
        Signals = Banker.tail(2)
        Signals = Signals.reset_index()

        Entry = (Signals.loc[0, 'Entry'] == False) & (Signals.loc[1, 'Entry'] == True)
        Last_Price = Signals.loc[1, 'Close']
        L1 = [hisse, Last_Price,Entry]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except:
        pass

df_True = df_signals[(df_signals['Dip Sinyali'] == True)]
df_True

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-poyw4_5m
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-poyw4_5m
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 574.8 kB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17532 sha256=6c52f3662cca8ba7c2a4b126294ffa5ad68f0b66f6d1205c752f977e45fb6680
  Stored in directory: /tmp/pip-ephem-wheel-cache-0qklcvym/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed


['A1CAP', 26.4, True]
['ACSEL', 110.5, False]
['ADEL', 548.5, False]
['ADESE', 2.0, True]
['ADGYO', 32.7, False]
['AEFES', 223.0, False]
['AFYON', 15.9, False]
['AGESA', 105.8, True]
['AGHOL', 338.75, False]
['AGROT', 16.27, False]
['AGYO', 7.16, False]
['AHGAZ', 13.43, False]
['AHSGY', 25.4, False]
['AKBNK', 61.15, False]
['AKCNS', 155.0, False]
['AKENR', 12.84, False]
['AKFGY', 2.25, False]
['AKFYE', 22.52, False]
['AKGRT', 5.87, False]
['AKMGY', 228.6, False]
['AKSA', 8.99, False]
['AKSEN', 39.54, False]
['AKSGY', 18.71, True]
['AKSUE', 11.41, False]
['AKYHO', 3.3, False]
['ALARK', 96.6, False]
['ALBRK', 6.11, False]
['ALCAR', 1139.0, False]
['ALCTL', 125.0, False]
['ALFAS', 58.45, True]
['ALGYO', 45.28, False]
['ALKA', 29.52, False]
['ALKIM', 33.76, False]
['ALKLC', 29.82, False]
['ALMAD', 7.11, False]
['ALTIN', 28.19, False]
['ALTNY', 100.7, False]
['ALVES', 49.16, False]
['ANELE', 12.86, False]
['ANGEN', 16.13, False]
['ANHYT', 103.9, False]
['ANSGR', 79.25, False]
['APBDL', 22.9

,Hisse Adı,Son Fiyat,Dip Sinyali
0,A1CAP,26.40,True
3,ADESE,2.00,True
7,AGESA,105.80,True
22,AKSGY,18.71,True
29,ALFAS,58.45,True
...,...,...,...
548,VESBE,17.87,True
552,VKING,45.66,True
555,YATAS,29.12,True
566,YUNSA,8.05,True
